In [3]:
position = 1

import pandas as pd
import numpy as np

pm=open('ppm_98_TMRend.fas','r')
npm=open('npm_98_TMRend.fas','r')

fas = [[]]

while True:
    reading = pm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_pm=pd.DataFrame(fas,columns=columns)

fas = [[]]

while True:
    reading = npm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_npm=pd.DataFrame(fas,columns=columns)


df_pm_freq=df_pm[position].value_counts(sort=True)
df_pm_nor=df_pm[position].value_counts(sort=True,normalize=True)

df_npm_freq=df_npm[position].value_counts(sort=True)
df_npm_nor=df_npm[position].value_counts(sort=True,normalize=True)

printout=pd.concat([df_pm_freq,df_pm_nor,df_npm_freq,df_npm_nor],axis='columns')

printout=printout.set_axis(['pm','pm_normalized','npm','npm_normalized'],axis='columns')

index1=printout.index

printout=printout.assign(AA=index1)

l_order = {'X': 0, 'L': 1, 'I': 2, 'V': 3,'F': 4, 'P': 5, 'M': 6, 'G': 7, 'A': 8, 'T': 9, 
         'W': 10,'K': 11,'H': 12,'R':13,'D': 14,'E':15, 'Q': 16,'N':17,'C': 18,'S':19,'Y': 20}

printout['order']=printout['AA'].map(l_order)

printout=printout.sort_values('order')

for i in range(20):
    if i != printout.iat[i,5]:
        
        for k,v in l_order.items():
            if i==v:
                kcap=k
        
        s=pd.DataFrame({'pm':[0],'pm_normalized':[0],
                        'npm':[0],'npm_normalized':[0],
                        'AA':kcap,'order':[i]},
                       index=[kcap])
        printout=printout.append(s)
        printout=printout.sort_values('order')

printout=printout.fillna(0)

printout['pm']=printout['pm'].astype('int')

# X抜きでnormalizedを出力

else_out=printout.drop('X')

sump=else_out['pm'].sum()
sumn=else_out['npm'].sum()

for i in range (19):
    nump=else_out.iat[i,0]
    numn=else_out.iat[i,2]
    
    else_out.iat[i,1]=100*nump/sump
    else_out.iat[i,3]=100*numn/sumn
    printout.iat[i,1]=100*printout.iat[i,1]
    printout.iat[i,3]=100*printout.iat[i,3]
    
    
else_out=else_out[['pm_normalized','npm_normalized']]

else_out=else_out.rename(columns={'pm_normalized': 'pm_Xout', 'npm_normalized': 'npm_Xout'})

printout=pd.concat([printout,else_out],axis='columns')

printout=printout.reindex(columns=['pm', 'npm', 'pm_normalized', 'npm_normalized',  'pm_Xout', 'npm_Xout'])

printout

,pm,npm,pm_normalized,npm_normalized,pm_Xout,npm_Xout
X,0,0,0.000000,0.000000,NaN,NaN
L,1,13,1.282051,2.309059,1.282051,2.309059
I,0,0,0.000000,0.000000,0.000000,0.000000
V,0,1,0.000000,0.177620,0.000000,0.177620
F,3,16,3.846154,2.841918,3.846154,2.841918
P,4,37,5.128205,6.571936,5.128205,6.571936
M,1,10,1.282051,1.776199,1.282051,1.776199
G,8,37,10.256410,6.571936,10.256410,6.571936
A,8,15,10.256410,2.664298,10.256410,2.664298
T,18,28,23.076923,4.973357,23.076923,4.973357
